In [1]:
import numpy as np

from typing import Tuple

In [2]:
DATA_PATH = './data'
example_X = np.load(f'{DATA_PATH}/example_X.npy')
example_treatment = np.load(f'{DATA_PATH}/example_treatment.npy')
example_y = np.load(f'{DATA_PATH}/example_y.npy')

example_preds = np.load(f'{DATA_PATH}/example_preds.npy')

In [3]:
example_X.shape

(50000, 5)

In [4]:
example_treatment, example_y

(array([1, 1, 1, ..., 1, 1, 0], dtype=int64),
 array([2.8480419 , 1.11098383, 0.24938254, ..., 3.88156137, 0.25859638,
        0.50738829]))

In [5]:
def criterion(
    treatment_target: np.ndarray,  # массив с целевыми значениями в целевой группе
    control_target: np.ndarray  # массив с целевыми значениями в контрольной группе
):
    """DeltaDeltaP"""
    return np.mean(treatment_target) - np.mean(control_target)

In [6]:
criterion(example_y[example_treatment.astype(bool)], example_y[~example_treatment.astype(bool)])

0.8133621067075112

In [7]:
def create_threshold_options(column_values: np.ndarray):
    unique_values = np.unique(column_values)
    if len(unique_values) > 10:
        percentiles = np.percentile(column_values, [3, 5, 10, 20, 30, 50, 70, 80, 90, 95, 97])
    else:
        percentiles = np.percentile(column_values, [10, 50, 90])
    threshold_options = np.unique(percentiles)
    return threshold_options

def find_threshold_to_split(
    column_values: np.ndarray,
    treatment: np.ndarray,
    target: np.ndarray
):
    threshold_options = create_threshold_options(column_values)
    best_threshold = None
    best_criterion = float('-inf')
    
    for option in threshold_options:
        condition = column_values <= option
        
        left_target, right_target = target[condition], target[~condition]
        left_treatment, right_treatment = treatment[condition], treatment[~condition]

        left_criterion = criterion(left_target[left_treatment.astype(bool)], left_target[~left_treatment.astype(bool)])
        right_criterion = criterion(right_target[right_treatment.astype(bool)], right_target[~right_treatment.astype(bool)])
        print('*'*30)
        print(option)
        print(left_criterion)
        print(right_criterion)
        print(len(left_target), len(right_target))
        criterion_value = np.abs(left_criterion - right_criterion)
        print(criterion_value)

        if criterion_value > best_criterion:
            best_threshold = option
            best_criterion = criterion_value
    return best_threshold, best_criterion

In [8]:
create_threshold_options(example_X[:, 0])

array([-1.88194354, -1.63390068, -1.27255591, -0.83430294, -0.52164123,
       -0.00405552,  0.52537453,  0.84283294,  1.28122757,  1.63985338,
        1.86980025])

In [9]:
find_threshold_to_split(example_X[:, 0], example_treatment, example_y)  # учесть гиперпараметры

******************************
-1.881943541589994
-1.4522051530430984
0.8813243356997235
1500 48500
2.3335294887428217
******************************
-1.633900676779844
-1.2349018002809096
0.9176303289378266
2500 47500
2.152532129218736
******************************
-1.272555905121884
-0.9253608754167951
1.0026398310581386
5000 45000
1.9280007064749336
******************************
-0.8343029365043767
-0.5815893044039584
1.1523727064437574
10000 40000
1.7339620108477158
******************************
-0.521641226833238
-0.3423012625893507
1.3012505801256338
15000 35000
1.6435518427149844
******************************
-0.0040555152405001
0.004014882972845135
1.6081859478386278
25000 25000
1.6041710648657825
******************************
0.5253745251188942
0.3079132817051238
1.9840915572446947
35000 15000
1.6761782755395709
******************************
0.8428329389786856
0.45661983599927736
2.2242964961219123
40000 10000
1.767676660122635
******************************
1.2812275747

(1.869800250289363, 2.3540019562549883)

In [10]:
example_y[~(example_X[:, 0]<=-1.88194354)]  # Так можно фильтровать массив по порогу значения

array([2.8480419 , 1.11098383, 0.24938254, ..., 3.88156137, 0.25859638,
       0.50738829])

In [11]:
np.abs(example_treatment-1)  # инверсия 0 и 1

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [12]:
class UpliftTreeRegressor:
    def __init__(
        self,
        max_depth: int = 3,  # максимальная глубина дерева
        min_samples_leaf: int = 1000,  # минимальное необходимое число обучающих объектов в листе дерева
        min_samples_leaf_treated: int = 300,  # минимальное необходимое число обучающих объектов с T=1 в листе дерева
        min_samples_leaf_control: int = 300  # минимальное необходимое число обучающих объектов с T=0 в листе дерева
    ) -> None:
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_leaf_treated = min_samples_leaf_treated
        self.min_samples_leaf_control = min_samples_leaf_control

        self.tree = {}

    @staticmethod
    def criterion(
        treatment_target: np.ndarray,  # массив с целевыми значениями в целевой группе
        control_target: np.ndarray  # массив с целевыми значениями в контрольной группе
    ) -> float:
        """DeltaDeltaP"""
        return np.mean(treatment_target) - np.mean(control_target)

    @staticmethod
    def create_threshold_options(column_values: np.ndarray) -> np.ndarray:
        '''Варианты порогов разбиения''' 
        unique_values = np.unique(column_values)
        if len(unique_values) > 10:
            percentiles = np.percentile(column_values, [3, 5, 10, 20, 30, 50, 70, 80, 90, 95, 97])
        else:
            percentiles = np.percentile(column_values, [10, 50, 90])
        threshold_options = np.unique(percentiles)
        return threshold_options
    
    def find_threshold_to_split(
        self,
        column_values: np.ndarray,
        treatment: np.ndarray,
        target: np.ndarray
    ) -> Tuple[float, float, float, float]:
        '''
        По заданному признаку ищем порог, который максимизирует критерий разбиения
        '''
        
        threshold_options = self.create_threshold_options(column_values)
        best_threshold = None
        best_criterion = float('-inf')
        best_left_criterion = None
        best_right_criterion = None
        
        for option in threshold_options:
            # проходим по всем порогам
            condition = column_values <= option  # условие разбиения, все данные, которые <= идут налево, остальные - направо
            
            left_target, right_target = target[condition], target[~condition]

            if len(left_target) < self.min_samples_leaf or len(right_target) < self.min_samples_leaf:
                # если число элементов в листе меньше минимального, то не рассматриваем этот порог
                continue
            
            left_treatment, right_treatment = treatment[condition], treatment[~condition]
            if sum(left_treatment) < self.min_samples_leaf_treated or sum(right_treatment) < self.min_samples_leaf_treated:
                # если число элементов, принадлежащих целевой группе в листе меньше минимального, то не рассматриваем этот порог
                continue

            if np.sum(np.abs(left_treatment-1)) < self.min_samples_leaf_control or np.sum(np.abs(right_treatment-1)) < self.min_samples_leaf_control:
                # если число элементов, принадлежащих контрольной группе в листе меньше минимального, то не рассмтариваем этот порог
                continue
    
            left_criterion = self.criterion(left_target[left_treatment.astype(bool)], left_target[~left_treatment.astype(bool)])
            right_criterion = self.criterion(right_target[right_treatment.astype(bool)], right_target[~right_treatment.astype(bool)])
 
            criterion_value = np.abs(left_criterion - right_criterion)

            if criterion_value > best_criterion:
                best_threshold = option
                best_criterion = criterion_value
                best_left_criterion = left_criterion
                best_right_criterion = right_criterion
                
        return best_threshold, best_criterion, best_left_criterion, best_right_criterion

    def find_split(
        self,
        X: np.ndarray,
        treatment: np.ndarray,
        y: np.ndarray
    ) -> Tuple[int, float, float, float, float]:
        '''
        Ищем лучший признак и его порог, что сделать разбиение
        '''
        
        num_features = X.shape[1]
        best_i = None
        best_threshold = None
        best_criterion = float('-inf')
        best_left_criterion, best_right_criterion = None, None

        # перебираем все признаки
        for i in range(num_features):
            threshold, criterion_value, left_criterion, right_criterion = self.find_threshold_to_split(X[:, i], treatment, y)
            if criterion_value > best_criterion:
                # если для текущего признака наша метрика максимальна, запишем его как лучший
                best_i = i
                best_threshold = threshold
                best_criterion = criterion_value
                best_left_criterion = left_criterion
                best_right_criterion = right_criterion
        return best_i, best_threshold, best_criterion, best_left_criterion, best_right_criterion

    def build(
        self,
        X: np.ndarray,
        treatment: np.ndarray,
        y: np.ndarray,
        depth,
        ATE,
        name
    ) -> dict:
        
        if depth > self.max_depth:
            # если достигли максимальной глубины - останавливаемся
            return

        tree = {}  # строим ноду
        tree['name'] = name
        tree['ATE'] = ATE  # значение ATE (average treatment effect) в ноде
        tree['n_items'] = len(X)  # число значений в ноде

        # ищем лучше разбиение
        best_i, best_threshold, best_criterion, best_left_criterion, best_right_criterion = self.find_split(X, treatment, y)

        tree['split_feat'] = best_i
        tree['split_threshold'] = best_threshold
        if best_i is None:
            # если по каким то условиям у нас не получилось найти лучшее разбиение (например - ограничение на число элементов в листе)
            return tree

        condition = X[:, best_i] <= best_threshold  # условие для разбиения на левую часть и правую
        left_X = X[condition, :]
        left_treatment = treatment[condition]
        left_y = y[condition]

        # сначала строим левую ветвь, используем рекурсию и наши новые найденные подмножества
        tree['left'] = self.build(left_X, left_treatment, left_y, depth+1, best_left_criterion, 'LEFT')

        right_X = X[~condition, :]
        right_treatment = treatment[~condition]
        right_y = y[~condition]

        # затем строим правую
        tree['right'] = self.build(right_X, right_treatment, right_y, depth+1, best_right_criterion, 'RIGHT')

        return tree
        
    def fit(
        self,
        X: np.ndarray,  # массив (n * k) с признаками
        treatment: np.ndarray,  # массив (n) с флагом воздействия
        y: np.ndarray  # массив (n) с целевой переменной
    ) -> None:
        
        root_ate = self.criterion(y[treatment.astype(bool)], y[~treatment.astype(bool)])  # исходный ATE в группах
        self.tree = self.build(X, treatment, y, 0, root_ate, 'ROOT')

    def predict_one(self, X: np.ndarray, node) -> float:
        '''
        Для предсказания одного элемента
        '''
        if node['split_feat'] is not None:  # это условие будет невыполнено только в случае терминального листа дерева
            if X[node['split_feat']] <= node['split_threshold']:  # идем налево, если <=
                if node['left'] is not None:  # если по условию разбиения у нас построилась нода, то спускаемся рекурсивно в нее
                    return self.predict_one(X, node['left'])
                else:
                    return node['ATE']
            else:  # аналогично для правой части
                if node['right'] is not None:
                    return self.predict_one(X, node['right'])
                else:
                    return node['ATE']
        else:
            return node['ATE']

    def predict(self, X: np.ndarray) -> np.ndarray:
        result = []
        # проходим по всем элементам выборки и для каждого элемента строим прогноз
        for i in range(len(X)):
            result.append(self.predict_one(X[i], self.tree))
        return np.array(result)

In [13]:
uplift_tree = UpliftTreeRegressor(max_depth=3, min_samples_leaf=6000, min_samples_leaf_treated=2500, min_samples_leaf_control=2500)

In [14]:
uplift_tree.fit(example_X, example_treatment, example_y)

In [15]:
uplift_tree.tree

{'name': 'ROOT',
 'ATE': 0.8133621067075112,
 'n_items': 50000,
 'split_feat': 0,
 'split_threshold': 0.8428329389786856,
 'left': {'name': 'LEFT',
  'ATE': 0.45661983599927736,
  'n_items': 40000,
  'split_feat': 0,
  'split_threshold': -0.9878097589516122,
  'left': {'name': 'LEFT',
   'ATE': -0.7089391259816358,
   'n_items': 8000,
   'split_feat': None,
   'split_threshold': None},
  'right': {'name': 'RIGHT',
   'ATE': 0.7439039464385158,
   'n_items': 32000,
   'split_feat': 1,
   'split_threshold': 0.8401218986161384,
   'left': {'name': 'LEFT',
    'ATE': 0.5381690182796833,
    'n_items': 25600,
    'split_feat': 0,
    'split_threshold': -0.377394679147196,
    'left': None,
    'right': None},
   'right': {'name': 'RIGHT',
    'ATE': 1.5773329275902146,
    'n_items': 6400,
    'split_feat': None,
    'split_threshold': None}}},
 'right': {'name': 'RIGHT',
  'ATE': 2.2242964961219123,
  'n_items': 10000,
  'split_feat': None,
  'split_threshold': None}}

In [19]:
with open(f'{DATA_PATH}/example_tree.txt', 'r') as f:
    print(f.read())

Root
n_items: 50000
ATE: 0.8133621067075059
split_feat: feat0
split_threshold: 0.8428329389786856

	Left
	n_items: 40000
	ATE: 0.45661983599928
	split_feat: feat0
	split_threshold: -0.9878097589516122

		Left <leaf>
		n_items: 8000
		ATE: -0.708939125981635
		split_feat: None
		split_threshold: None

		Right
		n_items: 32000
		ATE: 0.7439039464385134
		split_feat: feat1
		split_threshold: 0.8401218986161383

			Left <leaf>
			n_items: 25600
			ATE: 0.5381690182796856
			split_feat: None
			split_threshold: None

			Right <leaf>
			n_items: 6400
			ATE: 1.5773329275902113
			split_feat: None
			split_threshold: None

	Right <leaf>
	n_items: 10000
	ATE: 2.2242964961219096
	split_feat: None
	split_threshold: None


In [16]:
uplift_tree.predict(example_X)

array([ 0.53816902,  2.2242965 ,  0.53816902, ...,  1.57733293,
       -0.70893913,  2.2242965 ])

In [17]:
example_preds

array([ 0.53816902,  2.2242965 ,  0.53816902, ...,  1.57733293,
       -0.70893913,  2.2242965 ])

In [18]:
np.allclose(uplift_tree.predict(example_X), example_preds)

True